In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [2]:
dataset=pd.read_csv('PP_Mtrls_and_their_Mchncl_Prprts.csv')
dataset.head()

,Material,Su,Sy,E,G,mu,Ro,Use
0,ANSI Steel SAE 1015 asrolled,421,314,207000,79000,0.3,7860,True
1,ANSI Steel SAE 1015 normalized,424,324,207000,79000,0.3,7860,True
2,ANSI Steel SAE 1015 annealed,386,284,207000,79000,0.3,7860,True
3,ANSI Steel SAE 1020 asrolled,448,331,207000,79000,0.3,7860,True
4,ANSI Steel SAE 1020 normalized,441,346,207000,79000,0.3,7860,True


In [3]:
dataset.drop('Material', axis=1, inplace=True)

In [4]:
columns_to_drop = ['Sy','mu']
dataset = dataset.drop(columns=columns_to_drop)
dataset.head()

,Su,E,G,Ro,Use
0,421,207000,79000,7860,True
1,424,207000,79000,7860,True
2,386,207000,79000,7860,True
3,448,207000,79000,7860,True
4,441,207000,79000,7860,True


In [5]:
dataset['Use'] = dataset['Use'].astype('object')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 802 entries, 0 to 801
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Su      802 non-null    int64 
 1   E       802 non-null    int64 
 2   G       802 non-null    int64 
 3   Ro      802 non-null    int64 
 4   Use     802 non-null    object
dtypes: int64(4), object(1)
memory usage: 31.5+ KB


In [6]:
dataset=pd.get_dummies(dataset,drop_first=True)
dataset.head()

,Su,E,G,Ro,Use_True
0,421,207000,79000,7860,True
1,424,207000,79000,7860,True
2,386,207000,79000,7860,True
3,448,207000,79000,7860,True
4,441,207000,79000,7860,True


In [7]:
label_encoder = LabelEncoder()

# Fit and transform the target column
dataset['Use_True'] = label_encoder.fit_transform(dataset['Use_True'])
dataset

,Su,E,G,Ro,Use_True
0,421,207000,79000,7860,1
1,424,207000,79000,7860,1
2,386,207000,79000,7860,1
3,448,207000,79000,7860,1
4,441,207000,79000,7860,1
...,...,...,...,...,...
797,980,206000,80000,7860,0
798,1100,206000,80000,7860,0
799,1200,206000,80000,7860,0
800,1030,206000,80000,7860,0


In [8]:
indep=dataset[['Su','E','G','Ro']]
dep=dataset["Use_True"]

In [9]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [11]:
from sklearn.ensemble import RandomForestClassifier

In [12]:
from sklearn.model_selection import GridSearchCV
param_grid = {'criterion':['gini','entropy'],
              'max_features': ['auto','sqrt','log2'],
              'n_estimators':[10,100]} 
grid = GridSearchCV(RandomForestClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1') 
grid.fit(X_train, y_train) 

Fitting 5 folds for each of 12 candidates, totalling 60 fits


/Users/viswanathanmuthu/anaconda3/envs/AIVE_Course/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/viswanathanmuthu/anaconda3/envs/AIVE_Course/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/viswanathanmuthu/anaconda3/envs/AIVE_Course/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the pas

GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [10, 100]},
             scoring='f1', verbose=3)

In [13]:
re=grid.cv_results_
grid_predictions = grid.predict(X_test) 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[252   3]
 [  5   8]]


In [14]:
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.98      0.99      0.98       255
           1       0.73      0.62      0.67        13

    accuracy                           0.97       268
   macro avg       0.85      0.80      0.83       268
weighted avg       0.97      0.97      0.97       268



In [15]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'criterion': 'entropy', 'max_features': 'auto', 'n_estimators': 10}: 0.9689637748756218


In [16]:
from sklearn.metrics import roc_auc_score
roc_auc_score=roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])
print("The roc_auc_score is:",roc_auc_score)

The roc_auc_score is: 0.9923076923076922


In [17]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.014200,0.002529,0.003264,0.000671,gini,auto,10,"{'criterion': 'gini', 'max_features': 'auto', ...",0.600000,0.727273,0.666667,0.666667,0.600000,0.652121,0.047967,10
1,0.100679,0.004309,0.005468,0.002255,gini,auto,100,"{'criterion': 'gini', 'max_features': 'auto', ...",0.600000,0.727273,0.800000,0.666667,0.600000,0.678788,0.076948,5
2,0.013747,0.002340,0.001086,0.000171,gini,sqrt,10,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.285714,0.727273,0.800000,0.666667,0.571429,0.610216,0.178725,11
3,0.111229,0.019931,0.005582,0.003740,gini,sqrt,100,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.600000,0.727273,0.800000,0.666667,0.600000,0.678788,0.076948,5
4,0.014723,0.003266,0.002462,0.002152,gini,log2,10,"{'criterion': 'gini', 'max_features': 'log2', ...",0.250000,0.727273,0.800000,0.666667,0.600000,0.608788,0.191184,12
5,0.083767,0.008725,0.005422,0.002082,gini,log2,100,"{'criterion': 'gini', 'max_features': 'log2', ...",0.666667,0.727273,0.800000,0.666667,0.600000,0.692121,0.067314,4
6,0.010867,0.003188,0.001225,0.000664,entropy,auto,10,"{'criterion': 'entropy', 'max_features': 'auto...",0.600000,0.727273,0.833333,0.666667,0.750000,0.715455,0.078662,1
7,0.081275,0.003730,0.005395,0.003080,entropy,auto,100,"{'criterion': 'entropy', 'max_features': 'auto...",0.600000,0.727273,0.800000,0.666667,0.600000,0.678788,0.076948,5
8,0.011566,0.003239,0.001660,0.000779,entropy,sqrt,10,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.600000,0.666667,0.666667,0.666667,0.750000,0.670000,0.047610,9
9,0.076142,0.004089,0.003111,0.000484,entropy,sqrt,100,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.600000,0.727273,0.800000,0.666667,0.750000,0.708788,0.069237,2


In [18]:
su_input=float(input("Ultimate Tensile Strength (Su) in MPa:"))
E_input=float(input("Elastic Modulus (E) in MPa:"))
g_input=float(input("Shear Modulus (G) in MPa:"))
ro_input=float(input("Density (Ro) in Kg/m3:"))

Ultimate Tensile Strength (Su) in MPa:1000
Elastic Modulus (E) in MPa:119876
Shear Modulus (G) in MPa:67543
Density (Ro) in Kg/m3:8912


In [19]:
import pickle
filename="finalized_model_Random_Forest.sav"
pickle.dump(grid,open(filename,'wb'))

In [20]:
ppinput=sc.transform([[su_input,E_input,g_input,ro_input]])
ppinput

/Users/viswanathanmuthu/anaconda3/envs/AIVE_Course/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[ 1.01737213, -0.6495612 ,  0.01127766,  1.12379576]])

In [21]:
loaded_model=pickle.load(open("finalized_model_Random_Forest.sav",'rb'))
y=loaded_model.predict(ppinput)
if (y==0):
    print("Future_Prediction=[False]")
else:
    print('Future_Prediction=[True]')

Future_Prediction=[False]
[CV 3/5] END criterion=gini, max_features=auto, n_estimators=100;, score=0.800 total time=   0.1s
[CV 2/5] END criterion=gini, max_features=log2, n_estimators=10;, score=0.727 total time=   0.0s
[CV 5/5] END criterion=gini, max_features=log2, n_estimators=10;, score=0.600 total time=   0.0s
[CV 5/5] END criterion=gini, max_features=log2, n_estimators=100;, score=0.600 total time=   0.1s
[CV 5/5] END criterion=entropy, max_features=sqrt, n_estimators=10;, score=0.750 total time=   0.0s
[CV 3/5] END criterion=entropy, max_features=sqrt, n_estimators=100;, score=0.800 total time=   0.1s
[CV 4/5] END criterion=gini, max_features=auto, n_estimators=10;, score=0.667 total time=   0.0s
[CV 4/5] END criterion=gini, max_features=sqrt, n_estimators=10;, score=0.667 total time=   0.0s
[CV 2/5] END criterion=gini, max_features=sqrt, n_estimators=100;, score=0.727 total time=   0.1s
[CV 1/5] END criterion=entropy, max_features=auto, n_estimators=10;, score=0.600 total time

Future_Prediction=grid.predict([[su_input,E_input,g_input,ro_input]])
y=("Future_Prediction={}".format(Future_Prediction))
if (y==0):
    print("Future_Prediction=[False]")
else:
    print('Future_Prediction=[True]')